In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

import os
from dotenv import load_dotenv
load_dotenv()

Project_connection_string = os.getenv("Project_connection_string")